# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies
models.search_threshold = -1


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-3362000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-2082000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-

### Bidding

In [2]:
vuln_ns, vuln_ew = False, True
hand = 'A.8xx.Jxx.AQxxxx'

auction = ["1S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?


[{'call': '2C', 'insta_score': 0.976}]

In [3]:
auction = ["1S","PASS","1N","PASS","2C"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']


[{'call': '2H', 'insta_score': 0.493}]

In [4]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


West always has the same cards because it's your hand and it's known to you

In [5]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction,2)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

Good quality: True
xx.QJTx.AT8x.KTx KQ9xx.9xxx.Qxx.x A.8xx.Jxx.AQxxxx JT8xx.AK.K9x.J98 0.70
KQ8xx.K9xx.8x.T9 Jxxx.QT.KQxx.KJx A.8xx.Jxx.AQxxxx T9x.AJxx.AT9x.8x 0.75
KJx.QJ9x.Kxx.Jxx QT9xxx.AKx.T8.K8 A.8xx.Jxx.AQxxxx 8xx.Txx.AQ9xx.T9 0.75
9xxx.Q9x.9xxx.8x QJ8xx.AJx.KQx.Kx A.8xx.Jxx.AQxxxx KTx.KTxx.AT8.JT9 0.75
xxxx.J9xx.T9x.9x KJT9x.KTx.K8x.T8 A.8xx.Jxx.AQxxxx Q8x.AQx.AQxx.KJx 0.75
T9xx.QJxx.Axx.Jx Q8xx.9x.KTxx.KT9 A.8xx.Jxx.AQxxxx KJxx.AKTx.Q98.8x 0.74
QT9.T9x.KT9xx.8x K8xxx.AKxxx.AQ.9 A.8xx.Jxx.AQxxxx Jxxx.QJ.8xx.KJTx 0.72
QT98xx.ATx.KTx.J xx.QJxx.98xx.KT9 A.8xx.Jxx.AQxxxx KJxx.K9x.AQx.8xx 0.74
KQT8x.xx.9xx.T98 J9xx.AQJT.AKQx.J A.8xx.Jxx.AQxxxx xxx.K9xx.T8x.Kxx 0.75
Q98x.KJx.KQT9.JT KTxxx.AQxx.8xx.x A.8xx.Jxx.AQxxxx Jxx.T9x.Axx.K98x 0.74
